# CNN for HCM Classification

## Importing necessary dependencies

In [2]:
import random

from scipy import ndimage
from skimage.transform import rescale

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 


import PIL
print('Pillow Version:', PIL.__version__)
import glob
from matplotlib import image
import os
from os import listdir

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

## Importing Data

In [6]:
# load training images into directory
trainingpath = '/home/beepul/HCM-Project/ModifiedROI_4ChamberCine/Training'
loaded_images = list()
TopDirectory = listdir(trainingpath)

1

In [ ]:
# Load in training table
patientlist = []
training_df = pd.read_excel('/home/beepul/HCM-Project/HCM_Methods_Data/OrganizedData/Training.xls')
for patients in training_df['PatientNumber']:
    patientlist.append(str(patients))
training_df['PatientNumber'] = patientlist
training_df.head()

In [ ]:
# Create data frame to store relevant properties of each patient/image
num_patients = len(TopDirectory)
df = pd.DataFrame(columns = ['PatientNumber','Image','X_res','Y_res','X_length','Y_length','Outcome'])
df.PatientNumber = TopDirectory

# Load in each image and store in data frame
for i in range(num_patients):
    patient = df.PatientNumber[i]
    properties = training_df[training_df['PatientNumber'] == patient]
    index = properties.index[0]
    df.Outcome[i] = properties.Outcome[index]
    df.X_length[i], df.Y_length[i] = properties.X_Length[index], properties.Y_Length[index]
    df.X_res[i], df.Y_res[i] = properties.Xres[index], properties.Yres[index]
    imagepath = trainingpath + '/' + patient
    image_dir = listdir(imagepath)
    x_dim, y_dim = plt.imread(imagepath + '/' + str(1) + '.png').shape
    num_frames = len(image_dir)
    img_data = np.zeros((x_dim,y_dim,num_frames))
    for k in range(num_frames):
        framepath = imagepath + '/' + str(k+1) + '.png'
        img_data[:,:,(k)] = plt.imread(framepath)
    df.Image[i] = img_data

In [ ]:
# Load random montage to ensure the images loaded in correctly
volume = df.Image[3]
plt.figure(figsize=(20,10))
columns = 6
for i in range(volume.shape[2]):
    image = volume[:,:,i]
    plt.subplot(5, columns, i + 1)
    plt.imshow(image)

## Data Augmentation

In [ ]:
## Padding volumes to ensure sizes of images are correctly proportional to each other

# Find max_x and max_y
max_x_ind = pd.to_numeric(df.X_length).idxmax()
max_y_ind = pd.to_numeric(df.Y_length).idxmax()

In [ ]:
# Fine largest image within the training set
max_volume = df.iloc[max_x_ind]
max_y_length = max_volume['Y_length']
max_y_length

In [ ]:
# Function to properly pad images
# Input: Patient, desired x and y resolution
# Output: Resized and padded image
def pad(patient,d_x_res,d_y_res):
    image = patient.Image
    num_frames = image.shape[2]
    Y_length = patient.Y_length
    p = d_y_res*(Y_length/max_y_length)
    c_y_res = patient.Y_res
    scale = p/c_y_res
    resized_image = np.zeros((d_x_res,d_y_res,num_frames))
    for j in range(num_frames):
        frame = image[:,:,j]
        resized_frame = rescale(frame,scale)
        padded_frame = np.pad(resized_frame,((0,256-resized_frame.shape[0]),(0,256-resized_frame.shape[1])),'constant')
        resized_image[:,:,j] = padded_frame
    return resized_image

In [ ]:
# Load random montage to ensure the padding works correctly
patient = df.iloc[3]
resized_image = pad(patient,256,256)
volume = resized_image
plt.figure(figsize=(20,10))
columns = 6
for i in range(volume.shape[2]):
    image = volume[:,:,i]
    plt.subplot(5, columns, i + 1)
    plt.imshow(image)

In [ ]:
# Function to take images with 50 frames and reduce to 30 frames


In [ ]:
training_images = np.empty(df.shape[0])
training_labels = np.empty(df.shape[0])
for i in range(df.shape[0]):
    resized_image = pad(df.iloc[i],256,256)
    training_images[i] = resized_image
    training_labels[i] = df.Outcome[i]

## Encoder

In [15]:
# Currently Assuming that the input images are 150x150 with 30 frames
# Channel = 1 because they are grayscale
# This architecure is DEFINITELY subject to change

# Input Data
input_data = layers.Input([150,150,30,1])

# Conv 1.1
encoder = Conv3D(4,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(input_data)
encoder = BatchNormalization(center=True, scale=True)(encoder)
encoder = Conv3D(4,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(encoder)
encoder = BatchNormalization(center=True, scale=True)(encoder)
encoder = MaxPooling3D(pool_size = (2,2,2), strides=(2,2,2))(encoder)

# Conv 2.1
encoder = Conv3D(8,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(encoder)
encoder = BatchNormalization(center=True, scale=True)(encoder)
encoder = Conv3D(8,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(encoder)
encoder = BatchNormalization(center=True, scale=True)(encoder)
encoder = MaxPooling3D(pool_size = (3,3,3), strides=(3,3,3))(encoder)

# Conv 3.1
encoder = Conv3D(16,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(encoder)
encoder = BatchNormalization(center=True, scale=True)(encoder)
encoder = Conv3D(16,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(encoder)
encoder = BatchNormalization(center=True, scale=True)(encoder)
encoder = MaxPooling3D(pool_size = (3,3,3), strides=(5,5,5), padding = "same")(encoder)

# Size of data after convolutional layers
volumesize = tf.keras.backend.int_shape(encoder)

# Fully Connected
# 1st layer
encoder = Flatten()(encoder)
dense1_size = tf.keras.backend.int_shape(encoder)

# 2nd layer
encoder = layers.Dense(100)(encoder)

# Classifying layer
encoder_cnn = layers.Dense(2)(encoder)

In [9]:
# Summary of the encoder 
encoder_model = tf.keras.Model(input_data, encoder_cnn, name="encoder")
encoder_model.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 30, 1)] 0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 150, 150, 30, 4)   112       
_________________________________________________________________
batch_normalization_6 (Batch (None, 150, 150, 30, 4)   16        
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 150, 150, 30, 4)   436       
_________________________________________________________________
batch_normalization_7 (Batch (None, 150, 150, 30, 4)   16        
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 75, 75, 15, 4)     0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 75, 75, 15, 8)     872 

In [7]:


## Decoder
decoder = layers.Dense(dense1_size[1])(encoder)
decoder = layers.Reshape(volumesize[1:])(decoder)

# Deconv 1.1
decoder = Conv3D(16,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(decoder)
decoder = layers.BatchNormalization(center=True, scale=True)(decoder)
decoder = Conv3D(16,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(decoder)
decoder = layers.BatchNormalization(center=True, scale=True)(decoder)
decoder = layers.UpSampling3D([5,5,5])(decoder)

# Deconv 2.1
decoder = Conv3D(8,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(decoder)
decoder = layers.BatchNormalization(center=True, scale=True)(decoder)
decoder = Conv3D(8,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(decoder)
decoder = layers.BatchNormalization(center=True, scale=True)(decoder)
decoder = layers.UpSampling3D([3,3,3])(decoder)

# Deconv 2.1
decoder = Conv3D(4,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(decoder)
decoder = layers.BatchNormalization(center=True, scale=True)(decoder)
decoder = Conv3D(4,(3,3,3), strides=(1,1,1), activation="relu",padding = "same")(decoder)
decoder = layers.BatchNormalization(center=True, scale=True)(decoder)
decoder = layers.UpSampling3D([2,2,2])(decoder)
                 

In [170]:
# Decoder Summary

In [171]:
# Full Autoencoder Summary

autoencoder_model = tf.keras.Model(input_data, decoder, name="autoencoder")
autoencoder_model.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_48 (InputLayer)        [(None, 150, 150, 30, 1)] 0         
_________________________________________________________________
conv3d_216 (Conv3D)          (None, 150, 150, 30, 4)   112       
_________________________________________________________________
batch_normalization_228 (Bat (None, 150, 150, 30, 4)   16        
_________________________________________________________________
conv3d_217 (Conv3D)          (None, 150, 150, 30, 4)   436       
_________________________________________________________________
batch_normalization_229 (Bat (None, 150, 150, 30, 4)   16        
_________________________________________________________________
max_pooling3d_104 (MaxPoolin (None, 75, 75, 15, 4)     0         
_________________________________________________________________
conv3d_218 (Conv3D)          (None, 75, 75, 15, 8)     

In [172]:
input_img = tf.keras.Input(shape=(28, 28, 1))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(50)(x)
# at this point the representation is 50 dimensions

x = layers.Dense(128)(x)
x = layers.Reshape([4,4,8])(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

In [145]:
example = tf.keras.Model(input_img, decoded, name="autoencoder")
example.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 4, 4, 8)           